# Importing required Python libraries/packages

In [2]:
import getpass
import oracledb
import pandas as pd
from datetime import date

# Initialising **thick mode** for Oracle DB - for connection with the local DB instance.

In [3]:
oracledb.init_oracle_client(lib_dir='/opt/oracle/product/21c/dbhomeXE/lib')

# Establishing connection to the DB, as *SYS* user, in *SYSDBA* role

In [4]:
connection = oracledb.connect(
	mode=oracledb.AUTH_MODE_SYSDBA,
	dsn="localhost/XEPDB1"
)

# Creating a **cursor** to interact with the DB instance

In [5]:
cursor = connection.cursor()

# Executing, and displaying the output of, commands on the `placement_event` database

In [6]:
cursor.execute("select * from placement_event")

results = cursor.fetchall()
columns = [col[0] for col in cursor.description]

df = pd.DataFrame(results, columns=columns)
df

,EVENT_ID,EVENT_NAME,EVENT_DATE,EVENT_LOCATION,COMPANY_ID
0,1,Tech Job Fair,2024-03-10,Tech Park,1
1,2,Green Energy Conference,2024-04-12,Green Hall,2
2,3,Automotive Career Expo,2024-05-14,Auto Convention Center,3
3,4,Construction Expo,2024-06-16,Skyline Plaza,4
4,5,Healthcare Career Summit,2024-07-18,BioHealth Arena,5
5,6,Agriculture Conference,2024-08-20,AgroGrow Field,6
6,7,Fintech Career Fest,2024-09-22,Fintech Center,7


# Creating the tables

## Patient table

In [ ]:
# cursor.execute("drop table patient")

cursor.execute("""
create table patient(
    patient_id number primary key,
    name varchar2(100),
    address varchar2(255),
    date_of_birth date,
    phone varchar2(15)
    )
""")

## Doctor table

In [16]:
#cursor.execute("drop table doctor")

cursor.execute("""
create table doctor(
    doctor_id number primary key,
    name varchar2(100),
    specializations varchar2(255),
    experience_years number,
    contact varchar2(15)
    )
""")

## Room table

In [17]:
#cursor.execute("drop table room")

cursor.execute("""
create table room(
    room_id number primary key,
    room_type varchar2(50),
    capacity number,
    status varchar2(50)
)
""")

## Treatment table

In [18]:
#cursor.execute("drop table treatment")

cursor.execute("""
create table treatment(
    treatment_id number primary key,
    patient_id number,
    doctor_id number,
    treatment_type varchar2(100),
    start_date date,
    end_date date,
    foreign key(patient_id) references patient(patient_id),
    foreign key(doctor_id) references doctor(doctor_id)
)
""")

## Appointment table

In [20]:
# cursor.execute("drop table appointment")

cursor.execute("""
create table appointment(
    appointment_id number primary key,
    patient_id number,
    doctor_id number,
    room_id number,
    appointment_date date,
    appointment_time varchar2(10),
    foreign key(patient_id) references patient(patient_id),
    foreign key(doctor_id) references doctor(doctor_id),
    foreign key(room_id) references room(room_id)
)
""")

## Bill table

In [21]:
# cursor.execute("drop table bill")

cursor.execute("""
create table bill(
    bill_id number primary key,
    patient_id number,
    treatment_id number,
    total_amount number,
    date_issued date,
    foreign key(patient_id) references patient(patient_id),
    foreign key(treatment_id) references treatment(treatment_id)
)
""")

# Inserting records

## Patient table

In [8]:
patients = [
    (1, 'Aarav', '123, ABC St', date(1994, 5, 5), '1234567890'),
    (2, 'Priya', '45, DEF St', date(1995, 8, 12), '0987654321'),
    (3, 'Ravi', '78, GHI St', date(1994, 3, 3), '5647382910'),
    (4, 'Simran', '56, JKL St', date(1996, 2, 15), '4321567890')
]

cursor.executemany("""
    insert into patient values(:1, :2, :3, :4, :5)
    """, patients)

cursor.execute("select * from patient")
results = cursor.fetchall()
columns = [col[0] for col in cursor.description]

df = pd.DataFrame(results, columns=columns)
df

,PATIENT_ID,NAME,ADDRESS,DATE_OF_BIRTH,PHONE
0,1,Aarav,"123, ABC St",1994-05-05,1234567890
1,2,Priya,"45, DEF St",1995-08-12,0987654321
2,3,Ravi,"78, GHI St",1994-03-03,5647382910
3,4,Simran,"56, JKL St",1996-02-15,4321567890


## Doctor table

In [9]:
doctors = [
    (1, 'Dr. A', 'Cardiology', 15, '1231231234'),
    (2, 'Dr. B', 'Neurology', 10, '3213214321'),
    (3, 'Dr. C', 'Orthopedics', 20, '6546546543'),
    (4, 'Dr. D', 'Dermatology', 8, '7897897890')
]

cursor.executemany("""
    insert into doctor values(:1, :2, :3, :4, :5)
    """, doctors)

cursor.execute("select * from doctor")
results = cursor.fetchall()
columns = [col[0] for col in cursor.description]

df = pd.DataFrame(results, columns=columns)
df

,DOCTOR_ID,NAME,SPECIALIZATIONS,EXPERIENCE_YEARS,CONTACT
0,1,Dr. A,Cardiology,15,1231231234
1,2,Dr. B,Neurology,10,3213214321
2,3,Dr. C,Orthopedics,20,6546546543
3,4,Dr. D,Dermatology,8,7897897890


## Treatment table

In [10]:
treatments = [
    (1, 1, 1, 'Heart Surgery', date(2024, 1, 1), date(2024, 1, 10)),
    (2, 2, 2, 'Neurological Checkup', date(2024, 2, 5), date(2024, 2, 6)),
    (3, 3, 3, 'Orthopedic Surgery', date(2024, 3, 3), date(2024, 3, 10)),
    (4, 4, 4, 'Skin Treatment', date(2024, 4, 4), date(2024, 4, 8))
]

cursor.executemany("""
    insert into treatment values(:1, :2, :3, :4, :5, :6)
    """, treatments)

cursor.execute("select * from treatment")
results = cursor.fetchall()
columns = [col[0] for col in cursor.description]

df = pd.DataFrame(results, columns=columns)
df

,TREATMENT_ID,PATIENT_ID,DOCTOR_ID,TREATMENT_TYPE,START_DATE,END_DATE
0,1,1,1,Heart Surgery,2024-01-01,2024-01-10
1,2,2,2,Neurological Checkup,2024-02-05,2024-02-06
2,3,3,3,Orthopedic Surgery,2024-03-03,2024-03-10
3,4,4,4,Skin Treatment,2024-04-04,2024-04-08


## Room table

In [11]:
rooms = [
    (101, 'ICU', 2, 'Occupied'),
    (102, 'General', 4, 'Available'),
    (103, 'Private', 1, 'Available'),
    (104, 'General', 4, 'Occupied')
]

cursor.executemany("""
    insert into room values(:1, :2, :3, :4)
    """, rooms)

cursor.execute("select * from room")
results = cursor.fetchall()
columns = [col[0] for col in cursor.description]

df = pd.DataFrame(results, columns=columns)
df

,ROOM_ID,ROOM_TYPE,CAPACITY,STATUS
0,101,ICU,2,Occupied
1,102,General,4,Available
2,103,Private,1,Available
3,104,General,4,Occupied


## Appointment table

In [12]:
appointments = [
    (1, 1, 1, 101, date(2024, 1, 1), '10:00'),
    (2, 2, 2, 102, date(2024, 2, 5), '11:00'),
    (3, 3, 3, 103, date(2024, 3, 3), '12:00'),
    (4, 4, 4, 104, date(2024, 4, 4), '13:00')
]

cursor.executemany("""
    insert into appointment values(:1, :2, :3, :4, :5, :6)
    """, appointments)

cursor.execute("select * from appointment")
results = cursor.fetchall()
columns = [col[0] for col in cursor.description]

df = pd.DataFrame(results, columns=columns)
df

,APPOINTMENT_ID,PATIENT_ID,DOCTOR_ID,ROOM_ID,APPOINTMENT_DATE,APPOINTMENT_TIME
0,1,1,1,101,2024-01-01,10:00
1,2,2,2,102,2024-02-05,11:00
2,3,3,3,103,2024-03-03,12:00
3,4,4,4,104,2024-04-04,13:00


## Bill table

In [18]:
bills = [
    (1, 1, 1, 50000, date(2024, 1, 10)),
    (2, 2, 2, 15000, date(2024, 2, 6)),
    (3, 3, 3, 75000, date(2024, 3, 10)),
    (4, 4, 4, 10000, date(2024, 4, 8))
]

cursor.executemany("""
    insert into bill values(:1, :2, :3, :4, :5)
    """, bills)

cursor.execute("select * from bill")
results = cursor.fetchall()
columns = [col[0] for col in cursor.description]

df = pd.DataFrame(results, columns=columns)
df

,BILL_ID,PATIENT_ID,TREATMENT_ID,TOTAL_AMOUNT,DATE_ISSUED
0,1,1,1,50000,2024-01-10
1,2,2,2,15000,2024-02-06
2,3,3,3,75000,2024-03-10
3,4,4,4,10000,2024-04-08


# Closing up the cursor and connection

In [19]:
# connection.commit() - this way the program can be run each time to re-insert the records
# otherwise, the records will persist after the program is run once, so I cannot check easily.
cursor.close()
connection.close()

In [14]:
cursor.execute("alter session set constraints=deferred")
cursor.execute("drop table bill cascade constraints")
cursor.execute("drop table appointment cascade constraints")
cursor.execute("drop table treatment cascade constraints")
cursor.execute("drop table room cascade constraints")
cursor.execute("drop table doctor cascade constraints")
cursor.execute("drop table patient cascade constraints")